# This notebook will aim to use the trained models weights and load it on the inference model to generate predictions and novel sequences

In [94]:
from models.arcs import load_inf_model, GenPhiloText
from utilities.loaders import load_file
from utilities.preprocessors import preprocess, get_chars, map_value_to_index, decode_predictions
import tensorflow as tf

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load raw data

In [95]:
corpus = load_file('./data/notes.txt')
corpus = preprocess(corpus)
chars = get_chars(corpus)
char_to_idx = map_value_to_index(chars)
idx_to_char = map_value_to_index(chars, inverted=True)

In [96]:
char_to_idx.get_vocabulary()

['[UNK]',
 '\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '´',
 'ç',
 'é',
 'ï',
 '—',
 '…']

In [97]:
n_unique = len(char_to_idx.get_vocabulary())

# Declare same hyper params used in training model

In [98]:
emb_dim = 256
n_a = 1024
T_x = 100
dense_layers_dims = [n_unique] 
batch_size = 128
alpha = 1e-2
lambda_ = 0.8
drop_prob = 0.4
n_epochs =  100

# Declare sample input to build inference model and access .summary()

In [99]:
sample_input = tf.random.uniform(shape=(1, T_x), minval=0, maxval=n_unique - 1, dtype=tf.int32)
sample_h = tf.zeros(shape=(1, n_a))
sample_c = tf.zeros(shape=(1, n_a))

# Redeclare architecture by passing the same hyper params used in training and then load weights

In [100]:
saved_model = GenPhiloText(emb_dim=emb_dim, n_a=n_a, n_unique=n_unique, T_x=T_x, dense_layers_dims=dense_layers_dims)
saved_model([sample_input, sample_h, sample_c])

[<tf.Tensor: shape=(1, 57), dtype=float32, numpy=
 array([[ 1.2686631e-03, -2.6073668e-03, -2.9122133e-03,  3.6598677e-03,
          9.0519490e-04, -3.3376220e-04,  4.1950392e-03, -2.8621098e-03,
         -2.3476918e-04, -6.0864454e-03, -4.8256258e-04, -2.5532125e-03,
         -2.5469612e-03,  1.2792224e-05,  1.7397609e-03, -4.0597394e-03,
         -4.1276871e-07,  4.0977085e-03, -5.4093485e-04,  5.4823619e-04,
         -2.5439747e-03,  6.0331402e-04,  1.0346270e-03,  1.9346509e-03,
         -1.9838640e-03,  2.4329815e-03,  4.7664662e-04,  3.8824282e-03,
          2.0778941e-03,  2.0833204e-03,  1.5900466e-03, -2.7511131e-03,
         -2.7854750e-03, -3.8206603e-03,  3.0428232e-04,  1.4962484e-03,
          7.6039056e-03, -5.9746113e-03,  3.6396808e-03,  1.3075010e-03,
         -7.9246878e-04,  7.5732131e-04,  1.1447452e-03,  7.2555486e-03,
          2.1424299e-04, -1.0876947e-04,  2.8374072e-03,  2.7040561e-04,
         -2.5191114e-04, -1.7418744e-03,  1.1837278e-03, -6.8204640e-04,
 

In [101]:
saved_model.summary()

Model: "gen_philo_text_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 char-emb-layer (Embedding)  multiple                  14592     
                                                                 
 lstm-cell (LSTM)            multiple                  5246976   
                                                                 
 dense-layer-0 (Dense)       multiple                  58425     
                                                                 
 reshape-layer (Reshape)     multiple                  0         
                                                                 
Total params: 5319993 (20.29 MB)
Trainable params: 5319993 (20.29 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [102]:
saved_model.load_weights(filepath='./saved/weights/notes_gen_philo_text_21_295.9937.h5')
# saved_model.load_weights(filepath='./saved/weights/shakespeare_gen_philo_text_50_304.7047.h5')

# Extract trainable layers

In [103]:
layers = saved_model.layers
for layer in layers:
    print(layer.name)

char-emb-layer
lstm-cell
dense-layer-0
reshape-layer


In [104]:
char_emb_layer = saved_model.get_layer('char-emb-layer')
lstm_cell = saved_model.get_layer('lstm-cell')
dense_layers = []
norm_layers = []
for layer in layers:
    if "dense" in layer.name:
        dense_layers.append(saved_model.get_layer(layer.name))
    if "norm" in layer.name:
        norm_layers.append(saved_model.get_layer(layer.name))

In [105]:
dense_layers[0].get_weights()

[array([[ 0.00572441, -0.0031455 , -0.01837678, ...,  0.00292946,
          0.00171473,  0.00229066],
        [-0.00573133,  0.00321384,  0.00139457, ..., -0.00379671,
         -0.00169326, -0.00225277],
        [ 0.00567414, -0.00332178, -0.00057207, ...,  0.00258608,
          0.00159861,  0.00214501],
        ...,
        [ 0.00563983, -0.00331783,  0.0019683 , ...,  0.00364432,
          0.00159   ,  0.00214841],
        [ 0.0058742 , -0.0030037 , -0.03036108, ...,  0.00293198,
          0.00187656,  0.00248879],
        [ 0.00560772, -0.00330312, -0.00094625, ...,  0.00406584,
          0.00159263,  0.00211656]], dtype=float32),
 array([-0.4413016 , -0.08889223,  0.10745828, -0.42992154, -0.35343397,
        -0.4656473 , -0.45072484, -0.4170648 , -0.43667442, -0.05760063,
        -0.4272578 , -0.10096651, -0.42698967, -0.44869626, -0.44235805,
        -0.4203032 , -0.45736104, -0.45103905, -0.44370994, -0.44102904,
        -0.45489582, -0.4662597 , -0.4251941 , -0.39541736, -0.343

In [106]:
len(char_emb_layer.get_weights())

1

In [107]:
lstm_cell.get_weights()

[array([[-0.06083075, -0.03280047, -0.02655995, ..., -0.05329057,
         -0.05893561, -0.02346227],
        [ 0.04437323,  0.0751135 ,  0.02933332, ...,  0.01735416,
          0.06684623,  0.08056508],
        [-0.00902019,  0.01923245,  0.02827765, ..., -0.03634898,
         -0.09476296, -0.02848957],
        ...,
        [ 0.05324984,  0.03377243,  0.06852203, ...,  0.09527817,
          0.03429985,  0.07103755],
        [ 0.09136958,  0.00126808,  0.02201603, ...,  0.01121188,
          0.01381628,  0.03792499],
        [-0.09831004, -0.03199506, -0.05269979, ..., -0.07491152,
         -0.00288434, -0.02100489]], dtype=float32),
 array([[-0.05617283, -0.07053835, -0.07437034, ..., -0.06742737,
         -0.08059485, -0.04721952],
        [ 0.03056501,  0.03365387,  0.06721485, ...,  0.05210145,
          0.04700109,  0.05910198],
        [-0.0641828 , -0.07028978, -0.06563487, ..., -0.04569632,
         -0.06656666, -0.06028055],
        ...,
        [-0.04879535, -0.05956025, -0.0

# Build inference model

In [108]:
inference_model = load_inf_model(char_emb_layer, lstm_cell, dense_layers, char_to_idx=char_to_idx, T_x=250, temperature=1.2)

57
1024
tf.Tensor(
[[-inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]], shape=(1, 57), dtype=float32)
250
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189

In [109]:
inference_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 char-emb-layer (Embedding)  (None, 1, 256)               14592     ['input_7[0][0]',             
                                                                     'tf.random.categorical_1250[0
                                                                    ][0]',                        
                                                                     'tf.random.categorical_1251[0
                                                                    ][0]',                        
                                                                     'tf.random.categorical_

In [110]:
pred_ids = inference_model.predict([sample_input, sample_h, sample_c])
pred_ids

1/1 [==============================] - 166s 166s/step


[array([[6]], dtype=int64),
 array([[47]], dtype=int64),
 array([[55]], dtype=int64),
 array([[15]], dtype=int64),
 array([[33]], dtype=int64),
 array([[0]], dtype=int64),
 array([[39]], dtype=int64),
 array([[3]], dtype=int64),
 array([[13]], dtype=int64),
 array([[2]], dtype=int64),
 array([[20]], dtype=int64),
 array([[52]], dtype=int64),
 array([[27]], dtype=int64),
 array([[46]], dtype=int64),
 array([[47]], dtype=int64),
 array([[50]], dtype=int64),
 array([[32]], dtype=int64),
 array([[10]], dtype=int64),
 array([[35]], dtype=int64),
 array([[49]], dtype=int64),
 array([[38]], dtype=int64),
 array([[25]], dtype=int64),
 array([[1]], dtype=int64),
 array([[5]], dtype=int64),
 array([[49]], dtype=int64),
 array([[12]], dtype=int64),
 array([[19]], dtype=int64),
 array([[12]], dtype=int64),
 array([[35]], dtype=int64),
 array([[34]], dtype=int64),
 array([[16]], dtype=int64),
 array([[50]], dtype=int64),
 array([[13]], dtype=int64),
 array([[50]], dtype=int64),
 array([[41]], dtype

In [113]:
len(pred_ids)

250

In [111]:
decode_predictions(pred_ids, idx_to_char)

'\'w—2i[UNK]o!0 8çcvwzh-kyna\n&y/7/kj3z0zqpf…[UNK]ms.2[UNK]m7 1? esz6c3.lr rcsg0dé-\'8ïuliï[UNK](wïk(qkynij:lkglé,\'hx,ç0co7mn9x0n/3m—ï&ly\'st)v:m) j)k\' ?ï—sw,1tç…shp/a06…tm[UNK]:xbq;kugl…)17?çqwu; )i—,1(o2lvwgybqgh(-i)?!zpgsnégt—-[UNK]7ï?hk2n968!me/;"u"xay9z´"\'upbpf…l\n\n!3;logi,2'